In [ ]:
# default_exp HPsearch

In [ ]:
#export
from clearml import Task
from random import sample

TASK_NAME = "HP searching"
MY_PROJECT_NAME = "MY_PROJECT_NAME"


In [ ]:

# current_task_name = f"{TASK_NAME} - {datetime.now().strftime('%d-%m-%Y-%H-%M')}"

task = Task.init(project_name=MY_PROJECT_NAME, task_name=TASK_NAME, task_type=Task.TaskTypes.optimizer)

hp_options = dict()
hp_options = task.connect(hp_options)

hp_options.update({
    "batch_size": [25, 50, 100],
    "hidden_channels": [3, 5, 10],
    "num_layers": [1, 2, 3],
})

# noinspection PyDictCreation
hp_space = {
    'batch_size': lambda: sample(hp_options['batch_size'], 1)[0],
    'hidden_channels': lambda: sample(hp_options['hidden_channels'], 1)[0],
    'num_layers': lambda: sample(hp_options['num_layers'], 1)[0],
    # 'hidden_channels': lambda: sample(range(*hp_options['hidden_channels']), 1)[0],
    # 'num_layers': lambda: sample(range(*hp_options['num_layers']), 1)[0],
}
# number of random samples to test from 'space'
hp_options['total_number_of_experiments'] = 3

# execution queue to add experiments to
hp_options['execution_queue_name'] = 'default'

# experiment template to optimize with random parameter search
hp_options['experiment_template_name'] = TASK_NAME # TODO: try TASK_NAME/* instead

# Select base template task
# Notice we can be more imaginative and use task_id which will eliminate the need to use project name
template_task = Task.get_task(project_name=MY_PROJECT_NAME, task_name=hp_options['experiment_template_name'])
for i in range(hp_options['total_number_of_experiments']):
    # clone the template task into a new write enabled task (where we can change parameters)
    current_simulation_name =template_task.name+' {}'.format(i)
    cloned_task = Task.clone(source_task=template_task, name=current_simulation_name, parent=template_task.id)

    # get the original template parameters
    cloned_task_parameters = cloned_task.get_parameters()

    # override with random samples form grid
    for k in hp_space.keys():
        cloned_task_parameters[k] = hp_space[k]()

    # put back into the new cloned task
    cloned_task.set_parameters(cloned_task_parameters)
    print('Experiment {} set with parameters {}'.format(i, cloned_task_parameters))

    # enqueue the task for execution
    Task.enqueue(cloned_task.id, queue_name=hp_options['execution_queue_name'])
    print('Experiment id={} enqueue for execution'.format(cloned_task.id))
    # run_simulation(simulation_name=current_simulation_name,**cloned_task_parameters)

ClearML Task: created new task id=88c4a38670a443568623949727d6ff41
ClearML results page: https://app.clear.ml/projects/83a324bdf62d44b5acd422513c0fda30/experiments/88c4a38670a443568623949727d6ff41/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
2022-09-06 21:08:17,412 - clearml.util - WARNING - 2 task found when searching for `{'project_name': 'MY_PROJECT_NAME', 'task_name': 'HP searching', 'include_archived': True}`
2022-09-06 21:08:17,414 - clearml.util - WARNING - Selected task `HP searching` (id=88c4a38670a443568623949727d6ff41)
Experiment 0 set with parameters {'General/batch_size': '[25, 50, 100]', 'General/hidden_channels': '[3, 5, 10]', 'General/num_layers': '[1, 2, 3]', 'General/total_number_of_experiments': '3', 'General/execution_queue_name': 'default', 'General/experiment_template_name': 'HP searching', 'batch_size': 100, 'hidden_channels': 10, 'num_layers': 3}
Experiment id=bdd1f1a60fec4f1d88d07c1c8da58fc9 enqueue for exe